# Preparación e incorporación de datos demográficos de los países de la UE
En este notebook se extraen los datos demográficos de diferentes Excel y se ordenan de manera que puedan ser luego incorporados a los datasets pertinentes para, finalmente, ser incorporados.

In [1]:
import pandas as pd
import glob
import os

Primero leemos los excel y los limpiamos para generar .csv's con los que trabajar. Lo hacemos por separado para hombres y mujeres.

In [ ]:
ruta_carpeta = './Demographic_data/'
archivos_excel = glob.glob(os.path.join(ruta_carpeta, 'females*.xlsx'))

# Limpiar cada archivo
for archivo in archivos_excel:
    
    df = pd.read_excel(
        archivo,
        sheet_name='Sheet 1',
        skiprows=8,
        nrows=29
    )
    df = df[df.iloc[:, 0] != 'GEO (Labels)']  
    df.columns = df.iloc[0] 
    df = df[1:]
    df = df.loc[:, ~df.columns.isna()] 
    df.reset_index(drop=True, inplace=True) 
    df.columns = df.columns.astype(str).str.replace('.0', '', regex=False) 
    df = df.sort_values(by=df.columns[0]).reset_index(drop=True)

    # Guardar como csv
    nombre_salida = os.path.basename(archivo).replace('.xlsx', '.csv')
    ruta_salida = os.path.join(ruta_carpeta, nombre_salida)
    df.to_csv(ruta_salida, index=False)


In [ ]:
ruta_carpeta = './Demographic_data/'
archivos_excel = glob.glob(os.path.join(ruta_carpeta, 'males*.xlsx'))

# Limpiar cada archivo
for archivo in archivos_excel:
    
    df = pd.read_excel(
        archivo,
        sheet_name='Sheet 1',
        skiprows=8,
        nrows=29
    )
    df = df[df.iloc[:, 0] != 'GEO (Labels)']  
    df.columns = df.iloc[0] 
    df = df[1:]
    df = df.loc[:, ~df.columns.isna()] 
    df.reset_index(drop=True, inplace=True) 
    df.columns = df.columns.astype(str).str.replace('.0', '', regex=False) 
    df = df.sort_values(by=df.columns[0]).reset_index(drop=True)

    # Guardar como csv
    nombre_salida = os.path.basename(archivo).replace('.xlsx', '.csv')
    ruta_salida = os.path.join(ruta_carpeta, nombre_salida)
    df.to_csv(ruta_salida, index=False)

Ahora agrupamos los intervalos de edad en diferentes grupos que hemos considerado arbitrariamente adecuados para el estudio. Generamos dos .csv's finales, uno para cada sexo, donde está compendida toda la información.

In [32]:
ruta_carpeta = './Demographic_data/'
archivos_csv = glob.glob(os.path.join(ruta_carpeta, 'females_*.csv'))

grupos_edad = {
    'Fem. Niños (5-14 años)': ['5-9', '10-14'],
    'Fem. Adolescentes (15-19 años)': ['15-19'],
    'Fem. Adultos jóvenes (20-34 años)': ['20-24', '25-29', '30-34'],
    'Fem. Adultos medios (35-54 años)': ['35-39', '40-44', '45-49', '50-54'],
    'Fem. Adultos mayores (55-74 años)': ['55-59', '60-64', '65-69', '70-74'],
    'Fem. Ancianos (75+ años)': ['75-79', '80-84', '85+']
}

datos_agrupados = pd.DataFrame()

# Procesar cada grupo de edad
for grupo, franjas in grupos_edad.items():
    # Filtrar los archivos CSV correspondientes a las franjas de este grupo
    archivos_franjas = [f for f in archivos_csv if any(f"_{franja}_years.csv" in f for franja in franjas)]
    
    # Leer y combinar los datos de las franjas en este grupo
    df_grupo = pd.DataFrame()
    for archivo in archivos_franjas:
        df_franja = pd.read_csv(archivo)
        df_franja.rename(columns={df_franja.columns[0]: 'Identificador'}, inplace=True)
        df_grupo = pd.concat([df_grupo, df_franja])
    
    df_grupo = df_grupo.groupby('Identificador').sum().reset_index()
    df_grupo['Grupo de Edad'] = grupo
    
    datos_agrupados = pd.concat([datos_agrupados, df_grupo])

# Reorganizar las columnas para que el grupo de edad aparezca primero
columnas = ['Grupo de Edad'] + [col for col in datos_agrupados.columns if col != 'Grupo de Edad']
datos_agrupados = datos_agrupados[columnas]

# Guardar el resultado en un archivo CSV
archivo_salida = os.path.join(ruta_carpeta, 'poblacion_agrupada_females.csv')
datos_agrupados.to_csv(archivo_salida, index=False)


In [ ]:
ruta_carpeta = './Demographic_data/'
archivos_csv = glob.glob(os.path.join(ruta_carpeta, 'males_*.csv'))

grupos_edad = {
    'Masc. Niños (5-14 años)': ['5-9', '10-14'],
    'Masc. Adolescentes (15-19 años)': ['15-19'],
    'Masc. Adultos jóvenes (20-34 años)': ['20-24', '25-29', '30-34'],
    'Masc. Adultos medios (35-54 años)': ['35-39', '40-44', '45-49', '50-54'],
    'Masc. Adultos mayores (55-74 años)': ['55-59', '60-64', '65-69', '70-74'],
    'Masc. Ancianos (75+ años)': ['75-79', '80-84', '85+']
}

datos_agrupados = pd.DataFrame()

# Procesar cada grupo de edad
for grupo, franjas in grupos_edad.items():
    # Filtrar los archivos CSV correspondientes a las franjas de este grupo
    archivos_franjas = [f for f in archivos_csv if any(f"_{franja}_years.csv" in f for franja in franjas)]
    
    # Leer y combinar los datos de las franjas en este grupo
    df_grupo = pd.DataFrame()
    for archivo in archivos_franjas:
        df_franja = pd.read_csv(archivo)
        df_franja.rename(columns={df_franja.columns[0]: 'Identificador'}, inplace=True)
        df_grupo = pd.concat([df_grupo, df_franja])
    
    df_grupo = df_grupo.groupby('Identificador').sum().reset_index()
    df_grupo['Grupo de Edad'] = grupo
    
    datos_agrupados = pd.concat([datos_agrupados, df_grupo])

# Reorganizar las columnas para que el grupo de edad aparezca primero
columnas = ['Grupo de Edad'] + [col for col in datos_agrupados.columns if col != 'Grupo de Edad']
datos_agrupados = datos_agrupados[columnas]

# Guardar el resultado en un archivo CSV
archivo_salida = os.path.join(ruta_carpeta, 'poblacion_agrupada_males.csv')
datos_agrupados.to_csv(archivo_salida, index=False)


Por último juntamos los datos demográficos con los datos de consumo y climáticos que ya tenemos en otra carpeta, todo ello clasificado por país.

In [27]:
countries = ['austria','belgium','bulgaria','croatia','cyprus','czechia','denmark','estonia','finland','france','germany','greece','hungary',
             'ireland','italy','latvia','lithuania','luxembourg','malta','netherlands','poland','portugal','romania','slovenia','spain','sweden']

In [56]:
demo_data_fem = pd.read_csv('./Demographic_data/poblacion_agrupada_females.csv')
demo_data_masc = pd.read_csv('./Demographic_data/poblacion_agrupada_males.csv')

for country in countries:
    country_data = pd.read_csv(f'./Countries_citricos_clima/{country.capitalize()}.csv')
    
    # Datos demográficos femeninos
    demo_data_country_fem = demo_data_fem.loc[demo_data_fem['Identificador']==country.capitalize(),:]
    df_long_f = demo_data_country_fem.melt(id_vars=['Grupo de Edad', 'Identificador'], var_name='Año', value_name='Población')
    df_pivot_f = df_long_f.pivot(index='Año', columns='Grupo de Edad', values='Población')
    columnas_f = ['Fem. Niños (5-14 años)','Fem. Adolescentes (15-19 años)','Fem. Adultos jóvenes (20-34 años)','Fem. Adultos medios (35-54 años)',
            'Fem. Adultos mayores (55-74 años)','Fem. Ancianos (75+ años)']
    df_pivot_f = df_pivot_f[columnas_f]

    # Datos demográficos masculinos
    demo_data_country_masc = demo_data_masc.loc[demo_data_masc['Identificador']==country.capitalize(),:]
    df_long_m = demo_data_country_masc.melt(id_vars=['Grupo de Edad', 'Identificador'], var_name='Año', value_name='Población')
    df_pivot_m = df_long_m.pivot(index='Año', columns='Grupo de Edad', values='Población')
    columnas_m = ['Masc. Niños (5-14 años)','Masc. Adolescentes (15-19 años)','Masc. Adultos jóvenes (20-34 años)','Masc. Adultos medios (35-54 años)',
            'Masc. Adultos mayores (55-74 años)','Masc. Ancianos (75+ años)']
    df_pivot_m = df_pivot_m[columnas_m]

    # Fusión de los dataframes
    country_data['agno'] = country_data['agno'].astype(str)
    country_data.rename(columns={'agno': 'Año'}, inplace=True)

    df_merged_1 = country_data.merge(df_pivot_f, left_on="Año", right_on="Año", how="left")
    df_merged_2 = df_merged_1.merge(df_pivot_m, left_on="Año", right_on="Año", how="left")
    
    df_merged_2.drop(columns=['Week','mes'], inplace=True)
    df_merged_2.rename(columns={'Start date':'Date','p_mes':'Precipitación','temp_mes':'Temperatura'}, inplace=True)
 
    
    # Guardar los ficheros resultantes
    df_merged_2.to_csv(f'./Countries_citricos_completos/{country.capitalize()}.csv', index=False)
